In [25]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

In [26]:
form = transforms.Compose([transforms.Resize((227, 227), interpolation = transforms.InterpolationMode.BILINEAR), 
                           transforms.ToTensor()])

In [27]:
training_data = datasets.MNIST(root = "data", train = True, download = True, transform = form)

In [28]:
test_data = datasets.MNIST(root = "data", train = False, download = True, transform = form)

In [29]:
batch_size = 32

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([32, 1, 227, 227])
Shape of y: torch.Size([32]) torch.int64


In [30]:
device = "cpu" if not torch.cuda.is_available() else "cuda"
print(f"Using {device}")

Using cuda


In [49]:
class Neuralnetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        self.alexnet_stack = nn.Sequential(
            # 1 x 227 x 277
            nn.Conv2d(in_channels = 1   , out_channels = 96, kernel_size = 11, stride = 4, padding = 0),  #96 x 55 x 55
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2),   # 96 x 27 x27

            nn.Conv2d(in_channels = 96, out_channels = 256, kernel_size = 5, padding = 2, stride = 1), # 256 x 27 x27
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2), # 256 x13 x13

            nn.Conv2d(in_channels = 256, out_channels = 384, kernel_size = 3, padding = 1, stride = 1), # 384 x 13 x 13
            nn.ReLU(), 
            
            nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size = 3, padding = 1, stride = 1), # 384 x 13 x 13
            nn.ReLU(),

            nn.Conv2d(in_channels = 384, out_channels = 256, kernel_size = 3, padding = 1, stride = 1), # 256 x 13 x 13
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 3, stride = 2) # 256 x 6 x6
        )

        self.linear_stack = nn.Sequential(
            # nn.Flatten(),
            nn.Linear(in_features = 256 * 6 * 6, out_features = 4096),
            nn.ReLU(),

            nn.Linear(in_features = 4096, out_features = 4096),
            nn.ReLU(),

            nn.Linear(in_features = 4096, out_features = 10),
            nn.Softmax(dim = 1),
        )

    def forward(self, x):
        x = self.alexnet_stack(x)
        x = self.flatten(x)
        x = self.linear_stack(x)
        return x

In [50]:
model = Neuralnetwork().to(device)
print(model)

Neuralnetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (alexnet_stack): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_stack): Sequential(
    (0): Linear(in_features=9216, out_features=4096, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4096, out_features=4096, bias=True)
    (3): ReLU()
    (4): Lin

In [51]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

In [52]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 200 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [53]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [54]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 83.4%, Avg loss: 1.626574 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 70.4%, Avg loss: 1.757021 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 72.4%, Avg loss: 1.737203 

Epoch 4
-------------------------------


KeyboardInterrupt: 